In [ ]:
# Data Cleaning
import pandas as pd
df = pd.read_csv('bidding_data.csv')
group_cols = [col for col in df.columns if col not in ['bid_made', 'alone', 'score', 'hand_id']]
df_filtered = df.groupby(group_cols).filter(lambda g: len(g) > 2)
grouped_data = df_filtered.groupby(group_cols).apply(
    lambda x: pd.Series({
        'bid_win_rate': (x.loc[x['bid_made'] == 1, 'score']>0).mean(),
        'bid_expected_score': x.loc[x['bid_made'] == 1, 'score'].mean(),
        'not_bid_win_rate': (x.loc[x['bid_made'] == 0, 'score']>0).mean(),
        'not_bid_expected_score': x.loc[x['bid_made'] == 0, 'score'].mean(),
        'alone_win_rate': (x.loc[x['alone'] == 1, 'score']>0).mean(),
        'alone_expected_score': x.loc[x['alone'] == 1, 'score'].mean()
    })
)
grouped_data.to_csv("grouped.csv")
mask = ~(
    (

        
        grouped_data["bid_expected_score"].isna() & grouped_data["not_bid_expected_score"].between(-1, 1)
    ) | (
        grouped_data["not_bid_expected_score"].isna() & grouped_data["bid_expected_score"].between(-1, 1)
    )
)

# Keep rows that do NOT satisfy the condition
grouped_data_cleaned = grouped_data[mask]

def should_bid(row):
    a, b = row["bid_expected_score"], row["not_bid_expected_score"]
    
    # Case 1: both exist
    if pd.notna(a) and pd.notna(b):
        return int(a > b)
    
    # Case 2: only a exists
    if pd.notna(a) and pd.isna(b):
        if a > 1:
            return 1
        elif a < -1:
            return 0
    
    # Case 3: only b exists
    if pd.isna(a) and pd.notna(b):
        if b < -1:
            return 1
        elif b > 1:
            return 0
grouped_data_cleaned["should_bid"] = grouped_data_cleaned.apply(should_bid, axis=1)

def should_alone(row):
    a, b = row["bid_expected_score"], row["alone_expected_score"]

    if pd.notna(b) and (b > 1.5 or b > a):
        return 1

    else:
        return 0
grouped_data_cleaned["should_alone"] = grouped_data_cleaned.apply(should_alone, axis=1)

grouped_data_cleaned.to_csv("grouped_cleaned.csv")

In [ ]:
# Model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load data
df = pd.read_csv('grouped_cleaned.csv')

# Define features and targets
features = ['num_trump', 'right_bower', 'left_bower', 'trump_ace', 'trump_king', 
            'trump_queen', 'trump_ten', 'trump_nine', 'offsuit_aces', 'flipped_jack', 
            'flipped_ace', 'seat_position', 'bidding_round']
X = df[features]
y_bid = df['should_bid']
y_alone = df['should_alone']

# Split data (same splits for both targets)
X_train, X_test, _, _ = train_test_split(X, y_bid, test_size=0.2, random_state=42)
_, _, y_bid_train, y_bid_test = train_test_split(X, y_bid, test_size=0.2, random_state=42)
_, _, y_alone_train, y_alone_test = train_test_split(X, y_alone, test_size=0.2, random_state=42)

# Model Pipelines ---------------------------------------------------

# 1. BIDDING MODELS
bid_logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))
])

bid_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=42))
])

# 2. GOING ALONE MODELS
alone_logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))
])

alone_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=42))
])

# Training ----------------------------------------------------------

# Train bidding models
bid_logreg.fit(X_train, y_bid_train)
bid_rf.fit(X_train, y_bid_train)

# Train going-alone models
alone_logreg.fit(X_train, y_alone_train)
alone_rf.fit(X_train, y_alone_train)

# Evaluation --------------------------------------------------------

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    print(f"\n{model_name} Performance:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    return model

# Evaluate all models
bid_logreg = evaluate_model(bid_logreg, X_test, y_bid_test, "Bidding Logistic Regression")
bid_rf = evaluate_model(bid_rf, X_test, y_bid_test, "Bidding Random Forest")

alone_logreg = evaluate_model(alone_logreg, X_test, y_alone_test, "Going Alone Logistic Regression")
alone_rf = evaluate_model(alone_rf, X_test, y_alone_test, "Going Alone Random Forest")

# Prediction Example ------------------------------------------------

def make_prediction(model, input_data, model_name):
    sample_input = pd.DataFrame([input_data], columns=features)
    prediction = model.predict(sample_input)
    proba = model.predict_proba(sample_input)
    print(f"\n{model_name} Prediction:")
    print(f"Predicted class: {prediction[0]}")
    print(f"Probability estimates: {proba[0]}")
    return prediction

# Sample input: [num_trump, right_bower, left_bower, trump_ace, trump_king, 
#                trump_queen, trump_ten, trump_nine, offsuit_aces, flipped_jack, 
#                flipped_ace, seat_position, bidding_round]
sample_hand = [5, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0]

make_prediction(bid_logreg, sample_hand, "Bidding Logistic Regression")
make_prediction(bid_rf, sample_hand, "Bidding Random Forest")
make_prediction(alone_logreg, sample_hand, "Going Alone Logistic Regression")
make_prediction(alone_rf, sample_hand, "Going Alone Random Forest")

# Feature Importance ------------------------------------------------

print("\nBidding Random Forest Feature Importance:")
print(pd.DataFrame({
    'Feature': features,
    'Importance': bid_rf.named_steps['rf'].feature_importances_
}).sort_values('Importance', ascending=False))

print("\nGoing Alone Random Forest Feature Importance:")
print(pd.DataFrame({
    'Feature': features,
    'Importance': alone_rf.named_steps['rf'].feature_importances_
}).sort_values('Importance', ascending=False))

In [ ]:
# Test
from importlib import reload
from euchre import play_euchre

%reload_ext autoreload
%autoreload 2

winning_score = 2000

play_euchre(winning_score, player_strategies = ['new', 'scorecard_complex', 'new', 'scorecard_complex'], verbose=True)


In [ ]:
# Test
from importlib import reload
from euchre import play_euchre

%reload_ext autoreload
%autoreload 2

winning_score = 2000

play_euchre(winning_score, player_strategies = ['new', 'scorecard_simple', 'new', 'scorecard_simple'], verbose=True)
